In [1]:
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# https://www.analyticsvidhya.com/blog/2021/09/a-deep-dive-into-transformers-library/

In [3]:
from transformers import AutoTokenizer
checkpoint = "Hate-speech-CNERG/dehatebert-mono-polish"

In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=checkpoint)

In [5]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.3, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [6]:
raw_inputs = [
    "Błaszczak podkreślił, że będzie to zapora o wysokości 2,5 metra i szerokości 3 metrów. ", 
    "Panel obywatelski może być narzędziem wsparcia dla rządzących",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[  101,   100,   100,   117,   100,   100, 10114, 10354, 30954,   157,
           100,   123,   117,   126, 34439,   151,   100,   124,   100,   119,
           102],
        [  101,   100, 15547, 46908, 64167, 10523,   100,   100,   100,   165,
         65998, 10370, 11401, 12304,   100,   102,     0,     0,     0,     0,
             0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]])}


In [7]:
import torch

outputs = model(**inputs)
print(outputs.logits.shape)
outputs = torch.nn.functional.softmax(outputs.logits, dim = -1)
outputs

torch.Size([2, 2])


tensor([[0.9962, 0.0038],
        [0.9949, 0.0051]], grad_fn=<SoftmaxBackward0>)

In [8]:
model.config.id2label

{0: 'NON_HATE', 1: 'HATE'}

In [9]:
import pandas as pd

database_df = pd.read_csv("/content/drive/MyDrive/database.csv", lineterminator="\n")
database_df["hate"] = "UNKNOWN"
summary = database_df["summary"].apply(lambda x: str(x))
summary.fillna("")
summary

0       Jak zaznaczył, będą też prowadzone prace związ...
1       W tej chwili dynamicznie ta liczba wzrasta dod...
2       Jak go spytałem, za ile i co im sprzedał, powi...
3       Wniosek właściwie jest gotowy odpowiedział. Ja...
4       W takich sytuacjach ocenia się technologię, ko...
                              ...                        
6817     Polisę trzeba kupować z głową, tak by była do...
6818     Serwis ten daje możliwość skojarzenia tych, k...
6819     Wartości nie mają nic wspólnego z relacjami m...
6820    Wskazała, że takim niespodziewanym czynnikiem ...
6821     CZYTAJ RÓWNIEŻ: Premier podsumowuje COP26. Na...
Name: summary, Length: 6822, dtype: object

In [10]:
summary.tolist()[:5]

['Jak zaznaczył, będą też prowadzone prace związane z założeniem urządzeń, które pozwolą na elektroniczny dozór granicy. Jej wykonawcą jest Elektrotim. Koszt całej zapory na tej granicy to ok. 1,6 mld zł.',
 'W tej chwili dynamicznie ta liczba wzrasta dodał. Liczymy na 100 uczestników. Oni są rekrutowani zgodnie z najwyższymi standardami reprezentatywności.',
 'Jak go spytałem, za ile i co im sprzedał, powiedział, że sprzedał wszystko, a za ile, to mi nie powie zeznał. Nie pada tam ani jedno pytanie dotyczące sprzedaży nagrań” czytamy w publikacji Newsweeka. W poniedziałek konferencję prasową w związku z publikacją „Newsweeka” zorganizowali posłowie KO.',
 'Wniosek właściwie jest gotowy odpowiedział. Jak dodał, że ws. pieniędzy z funduszu spójności i polityki rolnej żadnych działań blokujących nie ma. Chcemy być przekonani, że interpretacja realizacji kamieni milowych jest taka sama jak nasza, a nie że ktoś będzie te same deklaracje interpretował w inny sposób” podkreślał.',
 'W takich

In [11]:
inputs = tokenizer(summary.tolist(), padding=True, truncation=True, return_tensors="pt")

In [12]:
len(inputs["token_type_ids"])

6822

In [ ]:
batch_size = 16
idx_prev = 0
from tqdm import tqdm
for idx in tqdm(range(batch_size, len(summary), batch_size)):
  inputs_batchs = {
      "input_ids": inputs["input_ids"][idx_prev:idx],
      "token_type_ids": inputs["token_type_ids"][idx_prev:idx],
      "attention_mask": inputs["attention_mask"][idx_prev:idx],
  }
  model.eval()
  outputs = model(**inputs_batchs)
  _, idecies_max = torch.max(outputs.logits, dim = -1)
  labels = [model.config.id2label[int(index_max.numpy())] for index_max in idecies_max]
  database_df.loc[idx_prev:idx-1, "hate"] = labels
  idx_prev = idx


 16%|█▌        | 68/426 [13:30<1:07:33, 11.32s/it]

In [ ]:
database_df.to_csv("/content/drive/MyDrive/database_with_hate.csv",)